# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [58]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [59]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[-1.1371,  1.4555],
         [-0.1576, -0.4780],
         [ 1.7980, -0.5389],
         [-0.0648, -1.5169],
         [-1.3706, -0.0802],
         [-0.0276, -0.7039],
         [ 0.5157, -0.6292],
         [ 0.1418,  0.0221],
         [ 0.2706,  0.4772],
         [-0.9251,  0.1379]]),
 tensor([[-3.0260],
         [ 5.5228],
         [ 9.6275],
         [ 9.2150],
         [ 1.7155],
         [ 6.5369],
         [ 7.3706],
         [ 4.4030],
         [ 3.1026],
         [ 1.8749]])]

使用框架的预定义好的层

In [60]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))#Sequential是一个存放神经网络层的容器

初始化模型参数
normal(0,0.01)方差为0，均值为0.01

In [61]:
net[0].weight.data.normal_(0, 0.01)#第0层中权重的数据替换为正态分布
net[0].bias.data.fill_(0)#偏差层的用权重用0替代

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [62]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [63]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
#torch.optim.SGD属于torch中优化器用来调整模型的参数，lr学习率

训练过程代码与我们从零开始实现时所做的非常相似

In [64]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        # 将trainer.zero_grad()放在l.backward()之前，
        # 可以确保在反向传播之前清零参数的梯度，
        # 以实现独立的梯度计算。
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000224
epoch 2, loss 0.000102
epoch 3, loss 0.000102


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [65]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0006, -0.0002])
b的估计误差： tensor([0.0009])
